In [ ]:
!pip install torchvision


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert images to tensor
])

# Create datasets
train_dataset = datasets.ImageFolder(root='/content/data/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/content/data/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:

def check_data_loader_dim(loader):
    for images, labels in loader:
        print('Image batch dimensions:', images.shape)
        print('Image label dimensions:', labels.shape)
        break

check_data_loader_dim(train_loader)
check_data_loader_dim(test_loader)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm.auto import tqdm
import gdown
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels[0], kernel_size=1)
        self.bn1 = nn.BatchNorm2d(out_channels[0])
        self.conv3 = nn.Conv2d(in_channels, out_channels[1], kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(out_channels[1])
        self.conv5 = nn.Conv2d(in_channels, out_channels[2], kernel_size=5, padding=2)
        self.bn5 = nn.BatchNorm2d(out_channels[2])
        self.conv_dil = nn.Conv2d(in_channels, out_channels[3], kernel_size=3, padding=2, dilation=2)
        self.bn_dil = nn.BatchNorm2d(out_channels[3])

    def forward(self, x):
        conv1 = F.relu(self.bn1(self.conv1(x)))
        conv3 = F.relu(self.bn3(self.conv3(x)))
        conv5 = F.relu(self.bn5(self.conv5(x)))
        conv_dil = F.relu(self.bn_dil(self.conv_dil(x)))
        return torch.cat([conv1, conv3, conv5, conv_dil], dim=1)


class DeeperMesoInception4(nn.Module):
    def __init__(self):
        super(DeeperMesoInception4, self).__init__()

        # Inception layer 1
        self.inception_1 = InceptionBlock(3, [1, 4, 4, 2])

        # Inception layer 2
        self.inception_2 = InceptionBlock(11, [2, 4, 4, 2])

        self.conv1 = nn.Conv2d(12, 16, kernel_size=5, padding=2)
        self.relu = nn.ReLU(inplace=True)
        self.bn = nn.BatchNorm2d(16)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.max_pool3 = nn.MaxPool2d(kernel_size=2)

        self.conv5 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.max_pool4 = nn.MaxPool2d(kernel_size=2)

        self.flatten = nn.Flatten()


        self.fc1 = nn.Linear(64 * 28 * 28, 16)  # Adjusted input size
        self.fc2 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.inception_1(x)
        x = self.inception_2(x)

        x = self.conv1(x)
        x = self.relu(x)
        x = self.bn(x)
        x = self.max_pool1(x)

        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.max_pool3(x)

        x = self.conv5(x)
        x = self.relu(x)
        x = self.conv6(x)
        x = self.relu(x)
        x = self.max_pool4(x)

        x = self.flatten(x)

        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x


In [ ]:
model = DeeperMesoInception4()
criterion = nn.BCEWithLogitsLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25, device='cpu'):
    model.to(device)
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training", unit="batch"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)

            labels = labels.unsqueeze(1).type(torch.float)  

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            preds = (outputs.sigmoid() > 0.5).float()  
            running_corrects += torch.sum(preds == labels)

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = running_corrects.double() / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        model.eval()
        running_loss = 0.0
        running_corrects = 0

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation", unit="batch"):
                inputs, labels = inputs.to(device), labels.to(device)
                labels = labels.unsqueeze(1).type(torch.float)  

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                preds = (outputs.sigmoid() > 0.5).float()
                running_corrects += torch.sum(preds == labels)

        epoch_val_loss = running_loss / len(val_loader.dataset)
        epoch_val_acc = running_corrects.double() / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        print(f'Epoch {epoch + 1}/{num_epochs}')
        print(f'Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}')
        print(f'Validation Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}')

    return model, (train_losses, train_accuracies, val_losses, val_accuracies)


In [ ]:
import torch
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Move the model to the correct device

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  

train_size = int(0.8 * len(train_dataset))  # 80% of the dataset for training
val_size = len(train_dataset) - train_size  # 20% for validation

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

trained_model, metrics = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=15, device=device)


In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from tqdm import tqdm

def evaluate_model(model, data_loader, device):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Evaluating", unit="batch"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            preds = torch.sigmoid(outputs).round()  # Assuming binary classification

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    conf_matrix = confusion_matrix(true_labels, predictions)

    return accuracy, f1, precision, recall, conf_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
accuracy, f1, precision, recall, conf_matrix = evaluate_model(trained_model, val_loader, device)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
